In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import mediapipe as mp
import cv2
import time
import matplotlib.pyplot as plt
import os

objc[42902]: Class CaptureDelegate is implemented in both /Users/nithiishkumarvijayadevan/mambaforge/envs/env_tensorflow/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x2831a0860) and /Users/nithiishkumarvijayadevan/mambaforge/envs/env_tensorflow/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x2bbe92538). One of the two will be used. Which one is undefined.
objc[42902]: Class CVWindow is implemented in both /Users/nithiishkumarvijayadevan/mambaforge/envs/env_tensorflow/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x28122ca68) and /Users/nithiishkumarvijayadevan/mambaforge/envs/env_tensorflow/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x2bbe92588). One of the two will be used. Which one is undefined.
objc[42902]: Class CVView is implemented in both /Users/nithiishkumarvijayadevan/mambaforge/envs/env_tensorflow/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x28122ca90) and /Users/nithiishkum

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils 

In [3]:
def mediapipe_detection(image, model):
    if image is not None:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
        image.flags.writeable = False                  # Image is no longer writeable
        results = model.process(image)                 # Make prediction
        image.flags.writeable = True                   # Image is now writeable 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
        return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) 
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) 
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [5]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
      
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [7]:
DATA_PATH = os.path.join('MP_Data9') 

no_sequences = 60

sequence_length = 40

actions = ['Thank you','Hackathon','Idle',"Hi","Jana Gana Mana Adhinayaka Jaya he'","Welcome"]
n = len(actions)

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [12]:
label_map = {label:num for num, label in enumerate(actions)}

In [13]:
label_map

{'Thank you': 0,
 'Hackathon': 1,
 'Idle': 2,
 'Hi': 3,
 "Jana Gana Mana Adhinayaka Jaya he'": 4,
 'Welcome': 5}

In [14]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [15]:
np.array(sequences).shape

(360, 40, 1662)

In [16]:
np.array(labels).shape

(360,)

In [17]:
X = np.array(sequences)

In [18]:
X.shape

(360, 40, 1662)

In [19]:
y = to_categorical(labels).astype(int)

In [20]:
y

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [22]:
y_test.shape


(36, 6)

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Bidirectional,GRU
from tensorflow.keras.callbacks import TensorBoard

In [24]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [26]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='tanh',input_shape=(40,1662)))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(LSTM(64, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(n, activation='softmax'))

Metal device set to: Apple M1


2022-08-23 12:18:48.287541: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-23 12:18:48.287879: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [27]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [28]:
model.fit(X_train, y_train, epochs=30, callbacks=[tb_callback])

Epoch 1/30


2022-08-23 12:18:52.495279: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-23 12:18:54.116801: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 12:18:54.398715: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 12:18:54.484697: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 12:18:54.541214: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 12:18:54.650492: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 12:18:54.780063: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113

11/11 [==============================] - 4s 60ms/step - loss: 1.7792 - categorical_accuracy: 0.2160
Epoch 2/30
11/11 [==============================] - 0s 34ms/step - loss: 1.3493 - categorical_accuracy: 0.4475
Epoch 3/30
11/11 [==============================] - 0s 32ms/step - loss: 1.1541 - categorical_accuracy: 0.4969
Epoch 4/30
11/11 [==============================] - 0s 33ms/step - loss: 0.8225 - categorical_accuracy: 0.7037
Epoch 5/30
11/11 [==============================] - 0s 33ms/step - loss: 0.7260 - categorical_accuracy: 0.7994
Epoch 6/30
11/11 [==============================] - 0s 34ms/step - loss: 0.6051 - categorical_accuracy: 0.7469
Epoch 7/30
11/11 [==============================] - 0s 33ms/step - loss: 0.4555 - categorical_accuracy: 0.7593
Epoch 8/30
11/11 [==============================] - 0s 33ms/step - loss: 0.4140 - categorical_accuracy: 0.8395
Epoch 9/30
11/11 [==============================] - 0s 33ms/step - loss: 0.3689 - categorical_accuracy: 0.8981
Epoch 10/30


In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 40, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 40, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 6)                 198       
                                                                 
Total params: 596,774
Trainable params: 596,774
Non-trai

In [30]:
model.save('sys.h5')

In [31]:
res = model.predict(X_test)

2022-08-23 12:19:12.107012: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 12:19:12.196437: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 12:19:12.246139: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-23 12:19:12.295634: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 105ms/step


In [32]:
actions[np.argmax(res[1])]

'Idle'

In [33]:
actions[np.argmax(y_test[1])]

'Idle'

In [41]:
sequence = []
sentence = []
threshold = 0.90

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.77, min_tracking_confidence=0.77) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        if frame is not None:
            # Make detections
            image, results = mediapipe_detection(frame, holistic)
            #print(results)
        
            # Draw landmarks
            draw_styled_landmarks(image, results)
        
            # 2. Prediction logic
            keypoints = extract_keypoints(results)
    #         sequence.insert(0,keypoints)
    #         sequence = sequence[:30]
            sequence.append(keypoints)
            sequence = sequence[-sequence_length:]
        
            if len(sequence) == sequence_length:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
            
            #3. Viz logic
                if res[np.argmax(res)] > threshold: 
                    #if actions[np.argmax(res)] == sentence[len(sentence)-1] and len(sentence) > 1:
                     #    pass
                    #else:
                        sentence.append(actions[np.argmax(res)])

                if len(sentence) > 1: 
                    sentence = sentence[-1:]
                #if len(sentence) > 3: 
                 #   sentence = sentence[-1:]
                    

                # Viz probabilities
                #image  = prob_viz(res, actions, image, colors)
            
            #cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,85), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (52,232,235), 2, cv2.LINE_AA)
        
            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully                                   
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 19ms/step


In [38]:
model.evaluate(X_test,y_test)

2022-08-22 11:22:29.427544: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-22 11:22:29.527474: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-22 11:22:29.579205: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/2 [==============>...............] - ETA: 0s - loss: 0.2242 - categorical_accuracy: 0.9688

2022-08-22 11:22:29.632842: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 129ms/step - loss: 0.1993 - categorical_accuracy: 0.9722


[0.1993401199579239, 0.9722222089767456]